In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import itertools
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely.wkt import loads
import warnings
from google.colab import drive
drive.mount('/content/drive')
TOM_FILES_DIR = "/content/drive/My Drive/Colab Notebooks/data science- transportation research/"
MAX_FILES_DIR = "/content/drive/My Drive/data science- transportation research/"
FILES_DIR = TOM_FILES_DIR

Mounted at /content/drive


In [4]:
STREET_FILE_NAME = "NYC Street Centerline (CSCL).geojson"
streets_df = gpd.read_file(f"{FILES_DIR}{STREET_FILE_NAME}")
streets_df = streets_df.reset_index(names="ST_INDEX")

In [ ]:
STREET_CLEANED_FILE_NAME = "cleaned_Centerline.csv"
streets_cleaned_df = pd.read_csv(f"{FILES_DIR}{STREET_CLEANED_FILE_NAME}")
streets_cleaned_df = streets_cleaned_df[["ST_INDEX",'With', 'Against', 'Two-Way', 'Non-Vehicular']]
streets_df = pd.merge(streets_df, streets_cleaned_df, on='ST_INDEX', how='inner')

In [68]:
df = streets_df.copy()
def get_start_end_of_street(sample):
    sample_location = sample["geometry"].boundary
    start = Point(sample_location.bounds[0:2])
    end = Point(sample_location.bounds[2:4])
    start_b = start.buffer(0.0001)
    end_b = end.buffer(0.0001)
    return start_b, end_b

df[["start", "end"]] = df.apply(get_start_end_of_street, axis=1, result_type='expand')
df["start"] = gpd.GeoSeries(df["start"])
df["end"] = gpd.GeoSeries(df["end"])


In [79]:
df_copy = df.copy()
points = ["start", "end"]
all_combinations = list(itertools.product(points, repeat=2))
global non_intersect_counter
non_intersect_counter = 0
global counter
counter = 0
def get_closest_street(sample):
    def get_one_intersection (first, second):
      sample_location = sample[first]
      df_streets_filtered = df_copy[df_copy[second].intersects(sample_location)]
      streets_index = df_streets_filtered["ST_INDEX"]
      set_streets_index = set(streets_index)
      return set_streets_index

    list_streets_index = set()
    for combination in all_combinations:
        point1, point2 = combination
        result_set = get_one_intersection(point1, point2)
        list_streets_index.update(result_set)
    list_streets_index = list(list_streets_index)
    if len(list_streets_index) == 1:
        global non_intersect_counter
        non_intersect_counter += 1

    global counter
    counter +=1
    if counter%1000==0:
      print(counter)
    padding_amount = 90-len(list_streets_index)
    if padding_amount<0:
      print(len(list_streets_index))
      print(sample["ST_INDEX"])
    return list_streets_index + [-1 for i in range(padding_amount)]

E_matrix = df.apply(get_closest_street, axis=1, result_type='expand')
# df['NGHBRS']
print(f"had {non_intersect_counter}"
      f" number of items with no intersection with a street.")

/usr/local/lib/python3.10/dist-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
had 24713 number of items with no intersection with a street.


In [80]:
E_matrix.to_csv("E_matrix_start_end.csv")

In [73]:
m = folium.Map(location=[40.7831, -73.9712], zoom_start=12)
folium.GeoJson(df["start"].iloc[1231].buffer(0.0001)).add_to(m)
for i in set_streets_index:
    # col1 = Point(small_collisions[["LONGITUDE", "LATITUDE"]].iloc[i:i + 1, ])
      folium.GeoJson(df["start"].iloc[i]).add_to(m)

m.save("map.html")
m

In [61]:
m = folium.Map(location=[40.7831, -73.9712], zoom_start=12)
folium.GeoJson(df["start"].iloc[1231].buffer(0.0001)).add_to(m)
folium.GeoJson(df["end"].iloc[1231].buffer(0.0001)).add_to(m)
for i in range(0,90):
    # col1 = Point(small_collisions[["LONGITUDE", "LATITUDE"]].iloc[i:i + 1, ])
    if E_matrix.iloc[1,i]!=-1:
      folium.GeoJson(df["start"].iloc[i]).add_to(m)
      folium.GeoJson(df["end"].iloc[i]).add_to(m)

m.save("map.html")
m

InvalidIndexError: ignored

In [ ]:
# df_copy = df.copy()
# def get_edges(sample):
#   if sample['Two-Way']==1 | sample['Non-Vehicular']==1:
#     df_streets_filtered = df_copy[streets_buffered_df["start"].intersects(sample["start"])]
#     if df_streets_filtered.shape[0] == 1:
#         global non_intersect_counter
#         non_intersect_counter += 1
#         print("another one")
#     streets_index = df_streets_filtered["ST_INDEX"]
#     list_streets_index = list(streets_index)

#     global counter
#     counter +=1
#     if counter%1000==0:
#       print(counter)
#     padding_amount = 50-len(list_streets_index)
#     if padding_amount<0:
#       print(len(list_streets_index))
#     return list_streets_index + [-1 for i in range(padding_amount)]

# df[["start", "end"]] = df.apply(get_edges, axis=1, result_type='expand')

# only by intersection, without direction

In [ ]:
small_streets_df = streets_df.iloc[:1000]
streets_buffered_df = small_streets_df.buffer(0.00015)

In [ ]:
df = small_streets_df.copy()
global non_intersect_counter
non_intersect_counter = 0
global counter
counter = 0
def get_closest_street(sample):
    sample_location = sample["geometry"]
    df_streets_filtered = small_streets_df[streets_buffered_df.intersects(sample_location)]
    if df_streets_filtered.shape[0] == 1:
        global non_intersect_counter
        non_intersect_counter += 1
        print("another one")

    streets_index = df_streets_filtered["index"]
    list_streets_index = list(streets_index)
    # result_series = pd.Series(0, index=range(50))
    # result_series.loc[streets_index] = 1

    global counter
    counter +=1
    if counter%1000==0:
      print(counter)
    padding_amount = 50-len(list_streets_index)
    if padding_amount<0:
      print(len(list_streets_index))
    return list_streets_index + [-1 for i in range(padding_amount)]

E_matrix = df.apply(get_closest_street, axis=1, result_type='expand')
# df['NGHBRS']
print(f"had {non_intersect_counter}"
      f" number of items with no intersection with a street.")


In [ ]:
sample_location = (small_streets_df.iloc[102696])["geometry"]
df_streets_filtered = small_streets_df[streets_buffered_df.intersects(sample_location)]
streets_index = df_streets_filtered["index"]
list(streets_index)

In [ ]:
m = folium.Map(location=[40.7831, -73.9712], zoom_start=12)
# for i in range(0,50):
    # col1 = Point(small_collisions[["LONGITUDE", "LATITUDE"]].iloc[i:i + 1, ])
    # if E_matrix.iloc[102696,i]!=-1:
    #   folium.GeoJson(streets_buffered_df.iloc[i]).add_to(m)
for i in list(streets_index):
  folium.GeoJson(small_streets_df["geometry"].iloc[i]).add_to(m)
folium.GeoJson(small_streets_df["geometry"].iloc[102696]).add_to(m)
m.save("map.html")
m